In [1]:
## Import Library ##
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

import tensorflow as tf
from tensorflow import keras
from sklearn.model_selection import train_test_split
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from keras.applications.inception_v3 import InceptionV3
from keras.layers import Dense, Dropout, GlobalAveragePooling2D, Input, Flatten
from keras.models import Model
from keras.callbacks import EarlyStopping, ReduceLROnPlateau, ModelCheckpoint

import os
import random
import warnings
warnings.filterwarnings('ignore')

In [2]:
## Seed ##
def seed_everything(seed):
    random.seed(seed)
    np.random.seed(seed)
    tf.random.set_seed(seed)
    os.environ['PYTHONHASHSEED'] = str(seed)
    os.environ['TF_DETERMINISTIC_OPS'] = '1'

seed = 42
seed_everything(seed)

In [3]:
## Load Data ##
folder_path = '../input/petfinder-pawpularity-score/'
train_file = '../input/petfinder-pawpularity-score/train.csv'
train_data = pd.read_csv(train_file)
train_data['path'] = train_data['Id'].map(lambda x: str(folder_path+'/train/'+x)+'.jpg')
print(train_data.shape)
train_data.head(3)

In [4]:
## Train Image Example ##
import matplotlib.pyplot as plt
import PIL

fig = plt.figure(figsize=(10,10))
path = train_data["path"][0]
img = PIL.Image.open(path)
plt.title("Pawpularity: 63")
plt.imshow(img)
plt.show()
fig.savefig("train_example.png")

In [5]:
## Train & valid Split ##
x_col = 'path'
y_col = 'Pawpularity'
test_size = 0.2

train, valid = train_test_split(train_data, test_size= test_size, random_state=seed, stratify=train_data[[y_col]])

In [6]:
## Transformation & Data Generator ##
img_width, img_height = 512, 512
batch_size = 32
no_of_classes = 1

train_datagen = ImageDataGenerator( preprocessing_function = None,
                                   horizontal_flip = True,
                                   vertical_flip = True)

train_generator = train_datagen.flow_from_dataframe(
    dataframe=train,x_col=x_col, y_col=y_col,
    target_size=(img_width, img_height),
    class_mode='raw',
    batch_size=batch_size,
    seed=seed,
    shuffle=True,
)

validation_datagen = ImageDataGenerator(preprocessing_function = None)
validation_generator = validation_datagen.flow_from_dataframe(
    dataframe=valid, x_col=x_col, y_col=y_col,
    target_size=(img_width, img_height),
    class_mode='raw',
    batch_size=batch_size,
    seed=seed,
    shuffle=True,
)

In [7]:
## Model ##
def create_model():
    input_tensor = Input(shape = (img_width, img_height,3))
    base_model = InceptionV3(weights = None, include_top = False, input_tensor = input_tensor)
    base_model.load_weights("../input/keras-applications-models/InceptionV3.h5")
    
    x = GlobalAveragePooling2D()(base_model.output)
    x = Flatten()(x)
    x = Dense(256, activation = 'relu')(x)
    x = Dropout(0.5)(x)
    final_output = Dense(no_of_classes)(x)
    
    model = Model(input_tensor, final_output)
    return model

In [8]:
## Training ##
model = create_model()

es = EarlyStopping(patience=5, monitor='val_mae', restore_best_weights=True)
rlrop = ReduceLROnPlateau(monitor='val_mae', mode='min', patience=3, factor=0.5, min_lr=1e-6, verbose=1)
cp = ModelCheckpoint('./inceptionv3_model.h5', verbose = 1, monitor = 'val_mae', mode = 'min', save_weights_only = True, save_best_only = True)
callback_list = [es, rlrop, cp]

optimizer = keras.optimizers.Adam(learning_rate=0.001, beta_1=0.9, beta_2=0.999, epsilon=None,  amsgrad=False)

model.compile(optimizer=optimizer,
              loss='mse',
              metrics=[tf.keras.metrics.RootMeanSquaredError(name="rmse"), "mae"])

In [9]:
model.fit_generator(train_generator,
                    epochs=30,
                    verbose=1,
                    validation_data=validation_generator,
                    callbacks = callback_list
                   )

In [ ]:
# from IPython.display import FileLink
# FileLink(r'./inceptionv3_model.h5')